## Top Level Understanding

In [1]:
import os
import openai

In [3]:
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
# api_key = os.environ.get("OPENAI_API_KEY")

client = openai.OpenAI(api_key = api_key)

In [49]:
from openai import OpenAI

client = OpenAI()
path = os.path.expanduser("~/Downloads/Amazon_V3_treated.wav")
# path = os.path.expanduser("~/Downloads/The Slightly Curious Studio 3 1-25m_x2_trimmed.m4a")
audio_file = open(path, "rb")

transcription = client.audio.transcriptions.create(
    model="gpt-4o-mini-transcribe", 
    # model = "whisper-1",
    file=audio_file, 
    response_format="json",
    stream = True
)

compiled_stream = ""
for event in transcription:
    if event.type == 'transcript.text.delta':
        compiled_stream += event.delta
        print(compiled_stream)


# client.audio.transcriptions.create(
#     model="gpt-4o-mini-transcribe", 
#     file=audio_file, 
#     # response_format="text",
#     stream = True
# )

Nature
Nature is
Nature is our
Nature is our greatest
Nature is our greatest ally
Nature is our greatest ally in
Nature is our greatest ally in the
Nature is our greatest ally in the fight
Nature is our greatest ally in the fight against
Nature is our greatest ally in the fight against climate
Nature is our greatest ally in the fight against climate change
Nature is our greatest ally in the fight against climate change.
Nature is our greatest ally in the fight against climate change. By
Nature is our greatest ally in the fight against climate change. By partnering
Nature is our greatest ally in the fight against climate change. By partnering with
Nature is our greatest ally in the fight against climate change. By partnering with NGOs
Nature is our greatest ally in the fight against climate change. By partnering with NGOs and
Nature is our greatest ally in the fight against climate change. By partnering with NGOs and experts
Nature is our greatest ally in the fight against climate chang

While this is all a nice idea, timestamps are more important and so maybe there is no place for this in the app? Or should I implement streaming where the user does not request timestamps?

In [50]:
import openai
from openai import OpenAI

client = OpenAI()

def stream_and_get_full_response(messages):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream=True
    )
    full_response_text = ""
    for chunk in response:
        if chunk.choices:
            delta = chunk.choices[0].delta
            if delta.content:
                full_response_text += delta.content
            if chunk.choices[0].finish_reason == "stop":
                # End of stream.  full_response_text now contains the full response.
                break
    return full_response_text

messages = [{"role": "user", "content": "Write a short story about a talking dog."}]
full_story = stream_and_get_full_response(messages)
print(full_story)

Once upon a sunny afternoon in the small town of Willowbrook, a curious little dog named Benny unearthed a peculiar object in his owner's backyard. Benny was a scruffy terrier with a heart full of adventure and a nose for trouble. As he scratched at the ground, he found an old, dusty lamp, half-buried beneath the roots of a gnarled oak tree.

With a wag of his tail, Benny pawed at the lamp until it rolled over, revealing a shiny side. Intrigued, he nudged it with his nose, and to his surprise, a soft, swirling mist poured out. It twisted and turned until it formed into a small, fluffy cloud. The cloud gently floated down and materialized into a tiny, talking genie, complete with a feathery blue hat and a long white beard.

“Greetings, noble pup!” the genie exclaimed, his voice melodious and booming. “I am Azar, the Genie of the Whimsical Wish! You have freed me from my slumber. In gratitude, I shall grant you one wish!”

Benny, absolutely baffled, tilted his head to the side. After a m

This doesn't seem to stream anything?

In [11]:
1400.501333/60

23.341688883333333

## Getting Langcodes

In [51]:
import pycountry 

countries = [country.alpha_2 for country in pycountry.countries]
# country_dict ={"Code" : country.alpha_2, "Country" : country.name for country in pycountry.countries}
country_dict ={country.alpha_2 : country.name for country in pycountry.countries}
country_dict_named = [
    {"label": name, "value": code}
    for code, name in country_dict.items()
]

country_dict_named

[{'label': 'Aruba', 'value': 'AW'},
 {'label': 'Afghanistan', 'value': 'AF'},
 {'label': 'Angola', 'value': 'AO'},
 {'label': 'Anguilla', 'value': 'AI'},
 {'label': 'Åland Islands', 'value': 'AX'},
 {'label': 'Albania', 'value': 'AL'},
 {'label': 'Andorra', 'value': 'AD'},
 {'label': 'United Arab Emirates', 'value': 'AE'},
 {'label': 'Argentina', 'value': 'AR'},
 {'label': 'Armenia', 'value': 'AM'},
 {'label': 'American Samoa', 'value': 'AS'},
 {'label': 'Antarctica', 'value': 'AQ'},
 {'label': 'French Southern Territories', 'value': 'TF'},
 {'label': 'Antigua and Barbuda', 'value': 'AG'},
 {'label': 'Australia', 'value': 'AU'},
 {'label': 'Austria', 'value': 'AT'},
 {'label': 'Azerbaijan', 'value': 'AZ'},
 {'label': 'Burundi', 'value': 'BI'},
 {'label': 'Belgium', 'value': 'BE'},
 {'label': 'Benin', 'value': 'BJ'},
 {'label': 'Bonaire, Sint Eustatius and Saba', 'value': 'BQ'},
 {'label': 'Burkina Faso', 'value': 'BF'},
 {'label': 'Bangladesh', 'value': 'BD'},
 {'label': 'Bulgaria', 'v

In [52]:
len(pycountry.countries)
list(pycountry.countries)[0]

Country(alpha_2='AW', alpha_3='ABW', flag='🇦🇼', name='Aruba', numeric='533')

In [23]:
import langcodes

dir(langcodes)

['ALL_SCRIPTS',
 'Any',
 'DEFAULT_LANGUAGE',
 'DEFAULT_SCRIPTS',
 'Dict',
 'Iterable',
 'LANGUAGE_ALPHA3',
 'LANGUAGE_ALPHA3_BIBLIOGRAPHIC',
 'LANGUAGE_NAME_IMPORT_MESSAGE',
 'LANGUAGE_REPLACEMENTS',
 'LIKELY_SUBTAGS',
 'Language',
 'LanguageData',
 'LanguageTagError',
 'List',
 'Mapping',
 'NORMALIZED_MACROLANGUAGES',
 'Optional',
 'Sequence',
 'TERRITORY_REPLACEMENTS',
 'Tuple',
 'Union',
 'VALIDITY',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'best_match',
 'closest_match',
 'closest_supported_match',
 'data_dicts',
 'find',
 'find_name',
 'get',
 'itemgetter',
 'language_distance',
 'normalize_characters',
 'parse_tag',
 'standardize_tag',
 'sys',
 'tag_distance',
 'tag_is_valid',
 'tag_match_score',
 'tag_parser',
 'tuple_distance_cached',
 'warnings']

# what about adding a prompt?

In [ ]:
transcription = client.audio.transcriptions.create(
    model="gpt-4o-mini-transcribe", 
    # model = "whisper-1",
    file=audio_file, 
    response_format="json",
    stream = True
)


## Trying chunking text

In [10]:
from pydub import AudioSegment

song = AudioSegment.from_mp3("good_morning.mp3")

# PyDub handles time in milliseconds
ten_minutes = 10 * 60 * 1000

first_10_minutes = song[:ten_minutes]

first_10_minutes.export("good_morning_10.mp3", format="mp3")

ModuleNotFoundError: No module named 'pydub'

In [54]:
import dash
import dash_bootstrap_components as dbc
from dash import html

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

card_content = [
    dbc.CardHeader("Card header"),
    dbc.CardBody(
        [
            html.H5("Card title", className="card-title"),
            html.P(
                "This is some card content that we'll reuse",
                className="card-text",
            ),
        ]
    ),
]

cards = html.Div(
    [
        dbc.Row(
            [
                dbc.Col(dbc.Card(card_content, color="primary", inverse=True)),
                dbc.Col(dbc.Card(card_content, color="secondary", inverse=True)),
                dbc.Col(dbc.Card(card_content, color="info", inverse=True)),
            ],
            className="mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(dbc.Card(card_content, color="success", inverse=True)),
                dbc.Col(dbc.Card(card_content, color="warning", inverse=True)),
                dbc.Col(dbc.Card(card_content, color="danger", inverse=True)),
            ],
            className="mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(dbc.Card(card_content, color="light")),
                dbc.Col(dbc.Card(card_content, color="dark", inverse=True)),
            ]
        ),
    ]
)

app.layout = html.Div(cards)

if __name__ == "__main__":
    app.run_server(debug=True)


nodename nor servname provided, or not known


SystemExit: 1

/Users/aoiferyan/Documents/tech_development/ShareScribe/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3675: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



# Translating to other languages

In [ ]:
response = client.responses.create(
    model="gpt-4o-mini",
    instructions=instructions,
    input="How would I declare a variable for a last name?",
)


In [ ]:
# Make sure requests package is installed  
import requests 
import os
import json

# Load the API key from the environment variable
api_key = os.getenv("OPENAI_API_KEY")


def process_audio_with_gpt_4o(base64_encoded_audio, output_modalities, system_prompt):
    # Chat Completions API end point 
    url = "https://api.openai.com/v1/chat/completions"

    # Set the headers
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    # Construct the request data
    data = {
        "model": "gpt-4o-mini",
        "modalities": output_modalities,
        "audio": {
            "voice": "alloy",
            "format": "wav"
        },
        "messages": [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "input_audio",
                        "input_audio": {
                            "data": base64_encoded_audio,
                            "format": "wav"
                        }
                    }
                ]
            }
        ]
    }
    
    request_response = requests.post(url, headers=headers, data=json.dumps(data))
    if request_response.status_code == 200:
        return request_response.json()
    else:  
        print(f"Error {request_response.status_code}: {request_response.text}")
        return
    